In [8]:
import sys
sys.path.append(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data_science_code')

from data_processor import DataProcessor
from classification_models import LogisticRegressionModel
from convert_categorical_variables import CategoricalEncoder

In [9]:
data_processor = DataProcessor()
train_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\train.csv')
test_df = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\test.csv')
gender_submission = data_processor.read_csv(r'C:\Users\huzef\OneDrive\Documents\Projects\Projects\data\titanic\gender_submission.csv')

In [10]:
train_data, test_data = data_processor.train_test_split(train_df, test_size=0.2, random_state=42)
X_train, y_train = data_processor.split_features_target(train_data, 'Survived')
X_test, y_test = data_processor.split_features_target(test_data, 'Survived')

Data split into training and testing sets successfully.
Features and target variable split successfully.
Features and target variable split successfully.


# EDA/ Feature Engineering

In [14]:
def eda(df_1):
    df = df_1.copy()
    df.drop('PassengerId', axis=1, inplace=True)

    df['Surname'] = df['Name'].str.split(',').str[0]
    df['FirstName'] = df['Name'].str.split(',').str[1]
    df['Title'] = df['FirstName'].str.split('.').str[0]
    df.drop(['Name', 'FirstName', 'Surname'], axis=1, inplace=True)
    rare_titles = ['Rev', 'Dr', 'Major', 'Col', 'Mlle', 'Capt', 'Mme', 'the Countess', 'Lady', 'Sir', 'Jonkheer', 'Don']
    df['Title'] = df['Title'].apply(lambda x: 'Miss' if x.strip() == 'Ms' else 'Rare' if x.strip() in rare_titles else x.strip())

    df['Sex'] = df['Sex'].map({'male': 1, 'female': 0})

    df['Ticket_string'] = df['Ticket'].str.contains('[a-zA-Z]').astype(int)

    df['Cabin'] = df['Cabin'].fillna('U')

    most_common = df['Embarked'].mode()[0]
    df['Embarked'] = df['Embarked'].fillna('most_common')

    df['Age'] = df['Age'].fillna(df['Age'].median())

    df['Cabin_Letter'] = df['Cabin'].str.extract(r'([A-Za-z])')
    df['Cabin_Number'] = df['Cabin'].str.extract(r'(\d+)')
    df['Cabin_Number'] = df['Cabin_Number'].fillna(0)
    
    df = df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Title','Ticket_string','Cabin_Letter','Cabin_Number']]

    encoder = CategoricalEncoder(df)
    df = encoder.one_hot_encoding(['Embarked', 'Title', 'Cabin_Letter'])

    return df

X_train_filt = eda(X_train)
X_test_filt = eda(X_test)

In [15]:
X_train_filt.head(30)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Ticket_string,Cabin_Letter,Cabin_Number
331,1,1,45.5,0,0,28.5000,S,Mr,0,C,124
733,2,1,23.0,0,0,13.0000,S,Mr,0,U,0
382,3,1,32.0,0,0,7.9250,S,Mr,1,U,0
704,3,1,26.0,1,0,7.8542,S,Mr,0,U,0
813,3,0,6.0,4,2,31.2750,S,Miss,0,U,0
...,...,...,...,...,...,...,...,...,...,...,...
106,3,0,21.0,0,0,7.6500,S,Miss,0,U,0
270,1,1,28.0,0,0,31.0000,S,Mr,0,U,0
860,3,1,41.0,2,0,14.1083,S,Mr,0,U,0
435,1,0,14.0,1,2,120.0000,S,Miss,0,B,96


In [ ]:
X_train = encoder.one_hot_encoding(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'])
X_train.head()

In [ ]:
log_reg_model = LogisticRegressionModel()
log_reg_model.set_params(random_state=42, max_iter=1000) 
log_reg_model.train(X_train, y_train)
print(log_reg_model.get_coefficients())
print(log_reg_model.get_params())
